In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [3]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [4]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer, PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import (train_test_split, cross_val_score, GridSearchCV, KFold, 
                                     StratifiedKFold, StratifiedShuffleSplit, RepeatedStratifiedKFold)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             classification_report, confusion_matrix, roc_curve, precision_recall_curve,
                             RocCurveDisplay, PrecisionRecallDisplay, ConfusionMatrixDisplay, 
                             make_scorer, average_precision_score, balanced_accuracy_score)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
# import category_encoders as ce

# from imblearn.over_sampling import SMOTE, ADASYN
# from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
# from imblearn.combine import SMOTEENN
# from collections import Counter

from sklearn.model_selection import cross_validate
from sklearn.model_selection import TunedThresholdClassifierCV, FixedThresholdClassifier

### Область работы 2 (выполнение лучшей модели)

In [6]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [7]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [8]:
# Блок обучения модели

In [9]:
df_test=pd.read_csv(path_test)
df_train=pd.read_csv(path_train)

In [10]:
df_train = df_train.drop_duplicates()

In [11]:
X = df_train.drop('species', axis=1)
y = df_train['species']

In [12]:
cat_features = ['island', 'sex']
axis_features = ['bill_depth_mm', 'bill_length_mm']
num_features = ['body_mass_g', 'flipper_length_mm', 'year']

In [13]:
axis_transformer = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=np.nan, max_iter=20)),
    ('polynom', PolynomialFeatures(2, include_bias=False)),  # Поставить True, если модель не имеет свободного члена
    ('scaler', MinMaxScaler())  # Это место будет заменяться скейлерами через GridSearchCV
])

# Пайплайн для числовых признаков
num = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=np.nan, max_iter=20)),
    ('scaler', MinMaxScaler())  # Здесь также заменяется скейлер
])

# Пайплайн для категориальных признаков
cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполнение пропусков наиболее частым значением
    ('onehot', OneHotEncoder(sparse_output=False))  # Кодирование категориальных переменных
])

# Объединение всех трансформеров в ColumnTransformer
CT = ColumnTransformer([
    ("axis_transformer", axis_transformer, axis_features),
    ("num", num, num_features),
    ("cat", cat, cat_features)
]).set_output(transform='pandas')

# Вывод информации о ColumnTransformer
display(CT)

ct = CT.fit_transform(df_train)

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('missing_num',
                                                  IterativeImputer(max_iter=20)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['bill_depth_mm', 'bill_length_mm']),
                                ('num',
                                 Pipeline(steps=[('missing_num',
                                                  IterativeImputer(max_iter=20)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['body_mass_g', 'flipper_length_mm', 'year']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['island', 'sex'])])

In [14]:
gnb = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', SVC(
        class_weight='balanced',
        probability=True,
        C=10,
        gamma='auto'
    ))
])

gnb.fit(X, y)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('axis_transformer',
                                                  Pipeline(steps=[('missing_num',
                                                                   IterativeImputer(max_iter=20)),
                                                                  ('polynom',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['bill_depth_mm',
                                                   'bill_length_mm']),
                                                 ('num',
                                                  Pipeline(steps=[('missing_num',
                                                                   IterativeImputer(max_iter=20)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['body_mass_g',
                                                   'flipper_length_mm',
                                                   'year']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['island', 'sex'])])),
                ('estimator',
                 SVC(C=10, class_weight='balanced', gamma='auto',
                     probability=True))])

In [15]:
# Блок предсказания с использованием тестового набора
y_predict = gnb.predict(df_test)

In [16]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
len(y_predict)

109

In [17]:
y_predict

array(['Adelie', 'Gentoo', 'Adelie', 'Chinstrap', 'Chinstrap', 'Gentoo',
       'Adelie', 'Chinstrap', 'Adelie', 'Adelie', 'Chinstrap', 'Gentoo',
       'Chinstrap', 'Gentoo', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie', 'Chinstrap',
       'Chinstrap', 'Adelie', 'Adelie', 'Chinstrap', 'Chinstrap',
       'Gentoo', 'Chinstrap', 'Chinstrap', 'Gentoo', 'Adelie', 'Adelie',
       'Adelie', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Adelie',
       'Gentoo', 'Chinstrap', 'Chinstrap', 'Adelie', 'Gentoo', 'Adelie',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Gentoo', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Adelie',
       'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Chinstrap', 'Chinstrap', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo',
       'Chinstrap', 'Gentoo', 'Gentoo', 'Gentoo', 'Gentoo', 'Chinstrap',
       'Gentoo', 'Gen